In [47]:
import os
import sys
from collections import defaultdict
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from util.CoNLL import readCoNLL, readOntoNotes


def convert_ontonotes_to_BIO(annotations) :
    current_tag = None
    inside_tag = False
    BIO_annotations = []
    for annotation in annotations:
        if annotation[0] == "(" and annotation[len(annotation) - 1] == ")" and current_tag is None :
            if current_tag is None :
                BIO_annotations.append("B-"+annotation[1:len(annotation)-1])
                #print("B-"+annotation[1:len(annotation)-1])
            else :
                print("WRONG ANNOTATION")
        elif annotation[0] == "(" :
            if current_tag is not None : 
                print("wrong annotation")
                raise("Error")
            else :
                current_tag = annotation[1: len(annotation)-1]
                BIO_annotations.append("B-"+current_tag)
                #print("Starting tag : {}".format(current_tag))
        elif annotation == "*" :
            if current_tag is not None:
                BIO_annotations.append("I-"+current_tag)
                #print("Currently inside : {}".format(current_tag))
            else :
                BIO_annotations.append("O")
                current_tag = None
                #print("Outside")
        elif annotation == "*)":
            if current_tag is not None :
                BIO_annotations.append("I-"+current_tag)
                current_tag = None
            else :
                print("WRONG ANNOTATION")
    #print(annotations)
    #print(BIO_annotations)
    #print("=======================================")
    return BIO_annotations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
sentences = readOntoNotes("/Users/slouvan/sandbox/cross-domain/data/conll-formatted-ontonotes-5.0/data/train/data/english/annotations/bc/cnn/00/cnn_0001.gold_conll",{3:'tokens', 10:'OntoNotes_Label'}, commentSymbol="#")

In [67]:
for sentence in sentences:
    annotations = sentence['OntoNotes_Label']
    BIO_annotations = convert_ontonotes_to_BIO(annotations)
    

In [68]:
convert_ontonotes_to_BIO(['(PER*', '*', '*)', '(ORG)','(LOC*','*','*', '*)', '*'])

['B-PER', 'I-PER', 'I-PER', 'B-ORG', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O']

In [75]:
import os
root_dir = "/Users/slouvan/sandbox/cross-domain/data/conll-formatted-ontonotes-5.0/data/test/data/english/annotations"
total_sentences = 0
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        current_file = os.path.join(subdir, file)
        if current_file.endswith(".gold_conll"):
            sentences = readOntoNotes(current_file,{3:'tokens', 10:'OntoNotes_Label'}, commentSymbol="#")
            for idx, sentence in enumerate(sentences):
                annotations = sentences[idx]['OntoNotes_Label']
                BIO_annotations = convert_ontonotes_to_BIO(annotations)
                sentences[idx]['OntoNotes_BIO'] = BIO_annotations
            
            with open(current_file+".bio", "w") as f:
                for sentence in sentences:
                    BIO_annotations = sentence['OntoNotes_BIO']
                    tokens = sentence['tokens']
                    for idx, token in enumerate(tokens):
                        f.write("{} {}\n".format(tokens[idx], BIO_annotations[idx]))
                    f.write("\n")

            total_sentences += len(sentences)
print(total_sentences)

12217
